In [36]:
# 모듈 로딩
import pandas as pd
import seaborn
import numpy as np

In [37]:
# Data 로딩
titanicDF = seaborn.load_dataset('titanic')
titanicDF

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [38]:
# 요약정보 확인
titanicDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [39]:
# 성별, 등석, 나이에 따른 생존률 구하기

# 새로운 데이터 프레임 생성
new_titanicDF = titanicDF[['sex', 'class', 'age', 'survived']].copy()

# 결측치 확인 및 처리방안 구상
new_titanicDF.isnull().sum()

sex           0
class         0
age         177
survived      0
dtype: int64

In [40]:
# age 컬럼에서 결측치 177개 관측, 평균값으로 채우기
new_titanicDF['age'].fillna(new_titanicDF['age'].mean(), inplace=True)

In [41]:
new_titanicDF['age'] = new_titanicDF['age'].astype(int)

In [42]:
# age 분류
bins = [0, 15, 30, 55, new_titanicDF['age'].max()]
new_titanicDF.insert(3, 'Age_boundary', pd.cut(new_titanicDF['age'], bins, 
                                               right=True, include_lowest=True,
                                               labels=['kids', 'youth', 'middle age', 'old age']))

In [43]:
# 최종 데이터 프레임 확인
new_titanicDF

,sex,class,age,Age_boundary,survived
0,male,Third,22,youth,0
1,female,First,38,middle age,1
2,female,Third,26,youth,1
3,female,First,35,middle age,1
4,male,Third,35,middle age,0
...,...,...,...,...,...
886,male,Second,27,youth,0
887,female,First,19,youth,1
888,female,Third,29,youth,0
889,male,First,26,youth,1


In [56]:
age_sv = np.round(new_titanicDF.groupby('Age_boundary')['survived'].mean(), 3)*100
sex_sv = np.round(new_titanicDF.groupby('sex')['survived'].mean(), 3)*100
sex_age_sv = np.round(new_titanicDF.groupby(['sex','Age_boundary'])['survived'].mean(), 3)*100
class_sv = np.round(new_titanicDF.groupby('class')['survived'].mean(),3)*100

print('===============연령대별 생존률(%)===============\n', 
      age_sv, '\n')

print('===============성별별 생존률(%)===============\n', 
      sex_sv, '\n')

print('===============성별+연령별 생존률(%)===============\n', 
      sex_age_sv, '\n')

print('===============객실별 생존률(%)===============\n', 
      class_sv, '\n')

===============연령대별 생존률(%)===============
 Age_boundary
kids          59.0
youth         33.5
middle age    42.4
old age       30.8
Name: survived, dtype: float64 

===============성별별 생존률(%)===============
 sex
female    74.2
male      18.9
Name: survived, dtype: float64 

===============성별+연령별 생존률(%)===============
 sex     Age_boundary
female  kids            65.1
        youth           72.2
        middle age      80.6
        old age         88.9
male    kids            52.5
        youth           14.0
        middle age      21.6
        old age         13.3
Name: survived, dtype: float64 

===============객실별 생존률(%)===============
 class
First     63.0
Second    47.3
Third     24.2
Name: survived, dtype: float64 

